<a href="https://colab.research.google.com/github/Janina712/RhythmMetrics_Spectral/blob/main/SpectralAnalysis_Example_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scipy

In [2]:
!pip install emd==0.6.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 7.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
import math
import statistics
import emd
from scipy import signal
from scipy.fft import fft, fftshift, fftfreq
import warnings
warnings.filterwarnings('ignore')

In [17]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/Grace/Aug_14_Files/

/content/gdrive/.shortcut-targets-by-id/1qo0RCBCZvuiciWPEdUPFQ2fR31kT1UeT/Aug_14_Files


**Load Data and Set-Up**

In [19]:
# participant IDs
IDs = ['NF1122']

In [20]:
conds = ['frog']

In [22]:
# load soundfiles
soundfiles = pd.DataFrame(columns=['ID', 'Condition', 'Fs', 'wav','t_wav'])
for ID in IDs:
  for cond in conds:
    Fs, wav = wavfile.read(ID + '_' + cond + '.wav')
    wav = wav/100000
    soundfiles = soundfiles._append({'ID': ID, 'Condition': cond, 'Fs': Fs, 'wav': wav}, ignore_index=True)

In [26]:
# load phrases
phrases = pd.DataFrame(columns=['ID', 'Condition', 'Onset','Offset','Duration', 'FluencyStatus','Signal'])
for ID in IDs:
  for cond in conds:
    names = []
    conditions = []
    duration = []
    utterances = pd.read_csv(ID + '_' + cond + '.txt', header=None, sep='\t')     # import txt
    fluencies = pd.read_csv(ID + '_FS_' + cond +  '.csv')                         # import csv
    names = list(np.repeat(ID, len(utterances)))                                  # ID column
    conditions = list(np.repeat(cond, len(utterances)))                           # condition column
    for k in range(len(utterances)):
        duration.append(float(utterances[3][k]) - float(utterances[2][k]))        # duration column
    participant = pd.DataFrame({'ID':names, 'Condition': conditions,'Onset': utterances[2], 'Offset': utterances[3], 'Duration':duration, 'FluencyStatus': fluencies['fluency status']})
    phrases = pd.concat([phrases, participant], ignore_index=True)

In [27]:
get_duration = []
for i in range(len(phrases)):
  if (phrases['Duration'][i] < 1.0) | (phrases['Duration'][i] > 3.0):
    continue
  else:
    get_duration.append(phrases["Duration"][i])

In [28]:
np.mean(get_duration)

2.1089276898897724

In [29]:
# compute time axis
for k in range(len(soundfiles)):
  t_wav = []
  for i in range(0,len(soundfiles['wav'][k])):
    time = i/Fs
    t_wav.append(time)
  soundfiles['t_wav'][k] = t_wav

In [30]:
# find where utterance onset is in t_wav
def find_nearest(array, value):
  array = np.asarray(array)
  idx = (np.abs(array - value)).argmin()
  return array[idx]

In [31]:
phrases_new = pd.DataFrame(columns=['ID', 'Condition', 'Onset','Offset','Duration', 'FluencyStatus','Signal'])
for cond in conds:
  soundfiles_subset = soundfiles[soundfiles["Condition"]==cond]
  soundfiles_subset.index = range(len(soundfiles_subset.index))
  phrases_subset = phrases[phrases["Condition"]==cond]
  phrases_subset.index = range(len(phrases_subset.index))
  for i in range(len(phrases_subset)):
    if phrases_subset['FluencyStatus'][i] == "Disfluent":
      continue
    else:
      if (phrases_subset['Duration'][i] < 1.0) |  (phrases_subset['Duration'][i] > 3.0):
        continue
      else:
        onset_time = find_nearest(soundfiles_subset['t_wav'][0], phrases_subset['Onset'][i])
        offset_time = find_nearest(soundfiles_subset['t_wav'][0], phrases_subset['Offset'][i])
        onset_idx = np.where(soundfiles_subset['t_wav'][0] == onset_time)
        onset_idx = list(onset_idx[0])[0]
        offset_idx = np.where(soundfiles_subset['t_wav'][0] == offset_time)
        offset_idx = list(offset_idx[0])[0]
        phrases_subset['Signal'][i] = list(soundfiles_subset['wav'][0][onset_idx:offset_idx])
        phrases_new = phrases_new._append({'ID':ID, 'Condition': cond, 'Onset': phrases['Onset'][i],'Offset': phrases['Offset'][i],'Duration': phrases['Duration'][i],'FluencyStatus': phrases['FluencyStatus'][i],'Signal':list(soundfiles_subset['wav'][0][onset_idx:offset_idx])}, ignore_index=True)

In [32]:
phrases_new

,ID,Condition,Onset,Offset,Duration,FluencyStatus,Signal
0,NF1122,frog,10.336740,12.585068,2.248328,fluent,"[-0.00116, -0.00114, -0.00121, -0.00124, -0.00..."
1,NF1122,frog,16.842627,19.208405,2.365778,fluent,"[0.00052, 0.00054, 0.00055, 0.00057, 0.00053, ..."
2,NF1122,frog,20.718476,23.319155,2.600678,fluent,"[-0.00138, -0.00146, -0.0014, -0.00142, -0.001..."
3,NF1122,frog,23.486940,26.104397,2.617457,fluent,"[0.00024, 0.00014, 0.00017, 9e-05, 0.0, 1e-05,..."
4,NF1122,frog,32.206169,35.075304,2.869135,fluent,"[-0.00052, -0.00056, -0.00057, -0.00058, -0.00..."
5,NF1122,frog,35.914232,38.716253,2.802021,fluent,"[-0.00032, -0.00027, -0.00028, -0.00019, -0.00..."
6,NF1122,frog,39.849853,42.786103,2.936250,fluent,"[-9e-05, -0.00019, -0.00028, -0.0003, -0.00041..."
7,NF1122,frog,43.759260,45.621681,1.862421,fluent,"[-0.00105, -0.00112, -0.00132, -0.00157, -0.00..."
8,NF1122,frog,46.217795,47.543302,1.325507,fluent,"[-0.0008, -0.00081, -0.00084, -0.0008, -0.0007..."
9,NF1122,frog,48.164109,50.345323,2.181214,fluent,"[0.00062, 0.00071, 0.0006, 0.00054, 0.00045, 0..."


# **I. Preprocessing**

In [33]:
# bandpass filter
lower_cutoff = 400
upper_cutoff = 4000
order = 4
bp_b, bp_a = signal.butter(order, [lower_cutoff, upper_cutoff], 'bp', fs=Fs)

In [34]:
# syllabic filter
order = 4
cutoff = 10
syll_b, syll_a = signal.butter(order, cutoff, 'lp', fs=Fs)

In [35]:
# down-sample parameters
original_rate = 44100
new_rate = original_rate/100

In [36]:
len(bp_a)

9

In [37]:
# pre-process
preprocessed_signals = pd.DataFrame(columns=['Duration', 'Signal'])
for i in range(len(phrases_new)):
  wav_dc = phrases_new['Signal'][i] - (sum(phrases_new['Signal'][i])/len(phrases_new['Signal'][i]))       # remove DC frequency
  filtered = signal.filtfilt(bp_b, bp_a, wav_dc)                            # bandpass filter
  new_filtered = abs(filtered)                                              # get magnitude (power)
  syll = signal.filtfilt(syll_b,syll_a, new_filtered)                       # apply syllabic filter
  sig_dc = syll - syll.mean()                                               # remove DC frequency (again)
  number_of_samples = round(len(sig_dc) * float(new_rate) /Fs)              # down-sample signal
  sig_down = signal.resample(sig_dc, number_of_samples)                     # down-sample signal
  avg2norm = (sum(sig_down))/len(sig_down)                                  # normalize
  intermediate = sig_down - avg2norm                                        # normalize
  norm_signal = intermediate/max(intermediate)                              # normalize
  M = len(norm_signal)                                                      # construct Tukey
  window = signal.windows.tukey(M, alpha = 0.2)                             # construct Tukey
  wind_signal = window*norm_signal                                          # apply Tukey
  preprocessed_signals = preprocessed_signals._append({'Duration': phrases_new['Duration'][i], 'Signal': wind_signal}, ignore_index=True)

# **II. Envelope Spectral Analysis (ESA)**

In [38]:
ESA_signals = pd.DataFrame(columns=['Duration', 'Signal','Time_Axis'])
for i in range(len(preprocessed_signals)):
  power_2 = 2**12                                                         # zero-pad
  to_add = int((power_2 - len(preprocessed_signals['Signal'][i])/2))
  right = np.append(preprocessed_signals['Signal'][i], [0]*to_add)
  left= np.insert(right, 0, [0]*to_add)
  zero_pad = left
  fourier = fft(zero_pad)                                                 # calculate amplitude spectrum
  xf = np.fft.fftfreq(len(zero_pad), (1/new_rate))
  power = (abs(fourier)**2)/len(fourier)                                  # get power
  norm_spec = power*2                                                     # double positive frequencies
  N = len(norm_spec)                                                      # cancel out negative frequencies
  power = np.abs(norm_spec[0:N//2])
  # adjust frequency axis
  T = 1/new_rate
  xf = fftfreq(N, T)[:N//2]
  power_sym = np.concatenate((np.flipud(power), power, np.flipud(power))) # center spectrum over 0 and N
  bin_size = 1                                                            # set parameters
  L = round(len(zero_pad)*bin_size/new_rate)
  smpsd = signal.lfilter((1/L)*np.ones(L),1,power_sym)                    # apply moving average
  smpsd = smpsd[int((N/2)+1):int((N/2)+(N/2))]
  ESA_signals = ESA_signals._append({'Duration': phrases_new['Duration'][i], 'Signal': smpsd,'Time_Axis': xf}, ignore_index=True)

# **III. Empirical Mode Decomposition**

In [39]:
def remove_outliers(IF):
  upper = IF.mean() + 3*IF.std()
  lower = IF.mean() - 3*IF.std()
  IF_no = IF
  for i in range(0,len(IF_no)):
    if IF[i] > upper:
      np.delete(IF_no,i, 0)
    elif IF[i] < lower:
      np.delete(IF_no,i,0)
    else:
      continue
  return IF_no

In [40]:
EMD_signals = pd.DataFrame(columns=['Duration', 'IMF1','IMF2', 'IA1','IA2'])
for i in range(len(preprocessed_signals)):
  imf = emd.sift.sift(preprocessed_signals['Signal'][i], max_imfs=4)                        # extract IMFs
  analytic_signal_1 = signal.hilbert(imf[:,0])                                              # Hilbert Transform
  analytic_signal_2 = signal.hilbert(imf[:,1])
  amplitude_envelope_1 = np.abs(analytic_signal_1)                                          # IA
  amplitude_envelope_2 = np.abs(analytic_signal_2)
  instantaneous_phase_1 = np.unwrap(np.angle(analytic_signal_1))                            # IP
  instantaneous_phase_2 = np.unwrap(np.angle(analytic_signal_2))
  instantaneous_frequency_1 = new_rate/(2.0 * np.pi) * np.gradient(instantaneous_phase_1)   # IF
  instantaneous_frequency_2 = new_rate/(2.0 * np.pi) * np.gradient(instantaneous_phase_2)
  IF_no_1 = remove_outliers(instantaneous_frequency_1)                                      # remove outliers
  IF_no_2 = remove_outliers(instantaneous_frequency_2)
  edge = round(new_rate/10)                                                                 # remove 100 ms from edges
  instantaneous_frequency_1_processed = IF_no_1[edge:len(IF_no_1)-edge]                     # remove edges from IFs
  instantaneous_frequency_2_processed = IF_no_2[edge:len(IF_no_2)-edge]
  amplitude_envelope_1_processed = amplitude_envelope_1[edge:len(IF_no_1)-edge]             # remove edges from IAs
  amplitude_envelope_2_processed = amplitude_envelope_2[edge:len(IF_no_2)-edge]
  EMD_signals = EMD_signals._append({'Duration': phrases_new['Duration'][i], 'IMF1': instantaneous_frequency_1_processed, 'IMF2': instantaneous_frequency_2_processed, 'IA1': amplitude_envelope_1_processed , 'IA2': amplitude_envelope_2_processed}, ignore_index=True)

# **IV. Rhythm Metrics**

In [41]:
metric_results = pd.DataFrame(columns=['Duration', 'SBPr','CNTR','IMFr','IMF1_rate','IMF2_rate','IMF1_stability','IMF2_stability'])
for i in range(len(ESA_signals)):
  supra_idx = np.where((ESA_signals['Time_Axis'][i] >= 1) & (ESA_signals['Time_Axis'][i] <= 3.5))[0]                # Spectral Band Power Ratio
  syll_idx = np.where((ESA_signals['Time_Axis'][i] > 3.5) & (ESA_signals['Time_Axis'][i] <= 10))[0]
  supra_power = []
  syll_power = []

  for w in range(0,len(supra_idx)):
    supra_power.append(ESA_signals['Signal'][i][supra_idx[w]])
  supra = sum(supra_power)

  for j in range(0,len(syll_idx)):
    syll_power.append(ESA_signals['Signal'][i][syll_idx[j]])
  syll = sum(syll_power)
  SBR_3_0 = supra/syll

  central_idx = np.where((ESA_signals['Time_Axis'][i] >= 1) & (ESA_signals['Time_Axis'][i] <= 10))[0]               # Envelope Spectral Centroid
  current_sum = 0
  for t in range(0,len(central_idx)):
    current_sum = current_sum + (ESA_signals['Time_Axis'][i][central_idx[t]] * ESA_signals['Signal'][i][central_idx[t]])
  power_sum = 0
  for m in range(0,len(central_idx)):
    power_sum = power_sum + ESA_signals['Signal'][i][central_idx[m]]
  CNTR = current_sum/power_sum

  imf1_power = []                                                 # IMF Ratio
  imf2_power = []

  for k in range(0,len(EMD_signals["IA1"][i])): #IMF1
    imf1_power.append(EMD_signals["IA1"][i][k])
  imf1 = sum(imf1_power)

  for j in range(0,len(EMD_signals["IA2"][i])):
    imf2_power.append(EMD_signals["IA2"][i][j])
  imf2 = sum(imf2_power)
  IMFr = imf2/imf1

  IMF1_rate = sum(EMD_signals["IMF1"][i])/len(EMD_signals["IMF1"][i])                         # IMF1 Rate
  IMF2_rate = sum(EMD_signals["IMF2"][i])/len(EMD_signals["IMF2"][i])                         # IMF2 Rate

  IMF1_stab = statistics.variance(EMD_signals["IMF1"][i])                      # IMF1 Stability
  IMF2_stab = statistics.variance(EMD_signals["IMF2"][i])                      # IMF2 Stability

  metric_results = metric_results._append({'Duration': phrases_new['Duration'][i], 'SBPr': SBR_3_0,'CNTR': CNTR,'IMFr': IMFr,'IMF1_rate': IMF1_rate,'IMF2_rate': IMF2_rate,'IMF1_stability': IMF1_stab,'IMF2_stability':IMF2_stab}, ignore_index = True)

In [42]:
print(IDs)
print(conds)

['NF1122']
['frog']


In [43]:
# mean across all utterances
result = metric_results.mean()
result = pd.DataFrame(result).T
result

,Duration,SBPr,CNTR,IMFr,IMF1_rate,IMF2_rate,IMF1_stability,IMF2_stability
0,2.108928,3.16603,2.988762,1.561236,4.710929,2.519324,35.829391,5.028479


In [44]:
# #utterances
len(metric_results)

16

**Loop for all subject IDs**

In [45]:
results_all = pd.DataFrame()

In [46]:
def remove_outliers(IF):
  upper = IF.mean() + 3*IF.std()
  lower = IF.mean() - 3*IF.std()
  IF_no = IF
  for i in range(0,len(IF_no)):
    if IF[i] > upper:
      np.delete(IF_no,i, 0)
    elif IF[i] < lower:
      np.delete(IF_no,i,0)
    else:
      continue
  return IF_no

In [48]:
IDs = ["NF1032",	"NF1082",	"NF1092",	"NF1114"]
conds = ['frog']

for ID in IDs:
    print(ID)
    for cond in conds:
        # load soundfiles
        soundfiles = pd.DataFrame(columns=['ID', 'Condition', 'Fs', 'wav','t_wav'])
        Fs, wav = wavfile.read(ID + '_' + cond + '.wav')
        wav = wav/100000
        soundfiles = soundfiles._append({'ID': ID, 'Condition': cond, 'Fs': Fs, 'wav': wav}, ignore_index=True)

        # load phrases
        phrases = pd.DataFrame(columns=['ID', 'Condition', 'Onset','Offset','Duration', 'FluencyStatus','Signal'])
        names = []
        conditions = []
        duration = []
        utterances = pd.read_csv(ID + '_' + cond + '.txt', header=None, sep='\t')     # import txt
        fluencies = pd.read_csv(ID + '_FS_' + cond +  '.csv')                         # import csv
        names = list(np.repeat(ID, len(utterances)))                                  # ID column
        conditions = list(np.repeat(cond, len(utterances)))                           # condition column
        for k in range(len(utterances)):
            duration.append(float(utterances[3][k]) - float(utterances[2][k]))        # duration column
        participant = pd.DataFrame({'ID':names, 'Condition': conditions,'Onset': utterances[2], 'Offset': utterances[3], 'Duration':duration, 'FluencyStatus': fluencies['fluency status']})
        phrases = pd.concat([phrases, participant], ignore_index=True)

        get_duration = []
        for i in range(len(phrases)):
          if (phrases['Duration'][i] < 1.0) | (phrases['Duration'][i] > 3.0):
            continue
          else:
            get_duration.append(phrases["Duration"][i])

        # compute time axis
        for k in range(len(soundfiles)):
          t_wav = []
          for i in range(0,len(soundfiles['wav'][k])):
            time = i/Fs
            t_wav.append(time)
          soundfiles['t_wav'][k] = t_wav

        # find where utterance onset is in t_wav
        def find_nearest(array, value):
          array = np.asarray(array)
          idx = (np.abs(array - value)).argmin()
          return array[idx]

        phrases_new = pd.DataFrame(columns=['ID', 'Condition', 'Onset','Offset','Duration', 'FluencyStatus','Signal'])
        for cond in conds:
          soundfiles_subset = soundfiles[soundfiles["Condition"]==cond]
          soundfiles_subset.index = range(len(soundfiles_subset.index))
          phrases_subset = phrases[phrases["Condition"]==cond]
          phrases_subset.index = range(len(phrases_subset.index))
          for i in range(len(phrases_subset)):
            if phrases_subset['FluencyStatus'][i] == "Disfluent":
              continue
            else:
              if (phrases_subset['Duration'][i] < 1.0) |  (phrases_subset['Duration'][i] > 3.0):
                continue
              else:
                onset_time = find_nearest(soundfiles_subset['t_wav'][0], phrases_subset['Onset'][i])
                offset_time = find_nearest(soundfiles_subset['t_wav'][0], phrases_subset['Offset'][i])
                onset_idx = np.where(soundfiles_subset['t_wav'][0] == onset_time)
                onset_idx = list(onset_idx[0])[0]
                offset_idx = np.where(soundfiles_subset['t_wav'][0] == offset_time)
                offset_idx = list(offset_idx[0])[0]
                phrases_subset['Signal'][i] = list(soundfiles_subset['wav'][0][onset_idx:offset_idx])
                phrases_new = phrases_new._append({'ID':ID, 'Condition': cond, 'Onset': phrases['Onset'][i],'Offset': phrases['Offset'][i],'Duration': phrases['Duration'][i],'FluencyStatus': phrases['FluencyStatus'][i],'Signal':list(soundfiles_subset['wav'][0][onset_idx:offset_idx])}, ignore_index=True)

        # parameters
        # bandpass filter
        lower_cutoff = 400
        upper_cutoff = 4000
        order = 4
        bp_b, bp_a = signal.butter(order, [lower_cutoff, upper_cutoff], 'bp', fs=Fs)

        # syllabic filter
        order = 4
        cutoff = 10
        syll_b, syll_a = signal.butter(order, cutoff, 'lp', fs=Fs)

        # down-sample parameters
        original_rate = 44100
        new_rate = original_rate/100

        # pre-processing
        preprocessed_signals = pd.DataFrame(columns=['Duration', 'Signal'])
        for i in range(len(phrases_new)):
          wav_dc = phrases_new['Signal'][i] - (sum(phrases_new['Signal'][i])/len(phrases_new['Signal'][i]))       # remove DC frequency
          filtered = signal.filtfilt(bp_b, bp_a, wav_dc)                            # bandpass filter
          new_filtered = abs(filtered)                                              # get magnitude (power)
          syll = signal.filtfilt(syll_b,syll_a, new_filtered)                       # apply syllabic filter
          sig_dc = syll - syll.mean()                                               # remove DC frequency (again)
          number_of_samples = round(len(sig_dc) * float(new_rate) /Fs)              # down-sample signal
          sig_down = signal.resample(sig_dc, number_of_samples)                     # down-sample signal
          avg2norm = (sum(sig_down))/len(sig_down)                                  # normalize
          intermediate = sig_down - avg2norm                                        # normalize
          norm_signal = intermediate/max(intermediate)                              # normalize
          M = len(norm_signal)                                                      # construct Tukey
          window = signal.windows.tukey(M, alpha = 0.2)                             # construct Tukey
          wind_signal = window*norm_signal                                          # apply Tukey
          preprocessed_signals = preprocessed_signals._append({'Duration': phrases_new['Duration'][i], 'Signal': wind_signal}, ignore_index=True)

        # ESA
        ESA_signals = pd.DataFrame(columns=['Duration', 'Signal','Time_Axis'])
        for i in range(len(preprocessed_signals)):
          power_2 = 2**12                                                         # zero-pad
          to_add = int((power_2 - len(preprocessed_signals['Signal'][i])/2))
          right = np.append(preprocessed_signals['Signal'][i], [0]*to_add)
          left= np.insert(right, 0, [0]*to_add)
          zero_pad = left
          fourier = fft(zero_pad)                                                 # calculate amplitude spectrum
          xf = np.fft.fftfreq(len(zero_pad), (1/new_rate))
          power = (abs(fourier)**2)/len(fourier)                                  # get power
          norm_spec = power*2                                                     # double positive frequencies
          N = len(norm_spec)                                                      # cancel out negative frequencies
          power = np.abs(norm_spec[0:N//2])
          # adjust frequency axis
          T = 1/new_rate
          xf = fftfreq(N, T)[:N//2]
          power_sym = np.concatenate((np.flipud(power), power, np.flipud(power))) # center spectrum over 0 and N
          bin_size = 1                                                            # set parameters
          L = round(len(zero_pad)*bin_size/new_rate)
          smpsd = signal.lfilter((1/L)*np.ones(L),1,power_sym)                    # apply moving average
          smpsd = smpsd[int((N/2)+1):int((N/2)+(N/2))]
          ESA_signals = ESA_signals._append({'Duration': phrases_new['Duration'][i], 'Signal': smpsd,'Time_Axis': xf}, ignore_index=True)

        # EMD
        EMD_signals = pd.DataFrame(columns=['Duration', 'IMF1','IMF2', 'IA1','IA2'])
        for i in range(len(preprocessed_signals)):
          imf = emd.sift.sift(preprocessed_signals['Signal'][i], max_imfs=4)                        # extract IMFs
          analytic_signal_1 = signal.hilbert(imf[:,0])                                              # Hilbert Transform
          analytic_signal_2 = signal.hilbert(imf[:,1])
          amplitude_envelope_1 = np.abs(analytic_signal_1)                                          # IA
          amplitude_envelope_2 = np.abs(analytic_signal_2)
          instantaneous_phase_1 = np.unwrap(np.angle(analytic_signal_1))                            # IP
          instantaneous_phase_2 = np.unwrap(np.angle(analytic_signal_2))
          instantaneous_frequency_1 = new_rate/(2.0 * np.pi) * np.gradient(instantaneous_phase_1)   # IF
          instantaneous_frequency_2 = new_rate/(2.0 * np.pi) * np.gradient(instantaneous_phase_2)
          IF_no_1 = remove_outliers(instantaneous_frequency_1)                                      # remove outliers
          IF_no_2 = remove_outliers(instantaneous_frequency_2)
          edge = round(new_rate/10)                                                                 # remove 100 ms from edges
          instantaneous_frequency_1_processed = IF_no_1[edge:len(IF_no_1)-edge]                     # remove edges from IFs
          instantaneous_frequency_2_processed = IF_no_2[edge:len(IF_no_2)-edge]
          amplitude_envelope_1_processed = amplitude_envelope_1[edge:len(IF_no_1)-edge]             # remove edges from IAs
          amplitude_envelope_2_processed = amplitude_envelope_2[edge:len(IF_no_2)-edge]
          EMD_signals = EMD_signals._append({'Duration': phrases_new['Duration'][i], 'IMF1': instantaneous_frequency_1_processed, 'IMF2': instantaneous_frequency_2_processed, 'IA1': amplitude_envelope_1_processed , 'IA2': amplitude_envelope_2_processed}, ignore_index=True)

        # compute metrics
        metric_results = pd.DataFrame(columns=['Duration', 'SBPr','CNTR','IMFr','IMF1_rate','IMF2_rate','IMF1_stability','IMF2_stability','#'])
        for i in range(len(ESA_signals)):
          supra_idx = np.where((ESA_signals['Time_Axis'][i] >= 1) & (ESA_signals['Time_Axis'][i] <= 3.5))[0]                # Spectral Band Power Ratio
          syll_idx = np.where((ESA_signals['Time_Axis'][i] > 3.5) & (ESA_signals['Time_Axis'][i] <= 10))[0]
          supra_power = []
          syll_power = []

          for w in range(0,len(supra_idx)):
            supra_power.append(ESA_signals['Signal'][i][supra_idx[w]])
          supra = sum(supra_power)

          for j in range(0,len(syll_idx)):
            syll_power.append(ESA_signals['Signal'][i][syll_idx[j]])
          syll = sum(syll_power)
          SBR_3_0 = supra/syll

          central_idx = np.where((ESA_signals['Time_Axis'][i] >= 1) & (ESA_signals['Time_Axis'][i] <= 10))[0]               # Envelope Spectral Centroid
          current_sum = 0
          for t in range(0,len(central_idx)):
            current_sum = current_sum + (ESA_signals['Time_Axis'][i][central_idx[t]] * ESA_signals['Signal'][i][central_idx[t]])
          power_sum = 0
          for m in range(0,len(central_idx)):
            power_sum = power_sum + ESA_signals['Signal'][i][central_idx[m]]
          CNTR = current_sum/power_sum

          imf1_power = []                                                 # IMF Ratio
          imf2_power = []

          for k in range(0,len(EMD_signals["IA1"][i])): #IMF1
            imf1_power.append(EMD_signals["IA1"][i][k])
          imf1 = sum(imf1_power)

          for j in range(0,len(EMD_signals["IA2"][i])):
            imf2_power.append(EMD_signals["IA2"][i][j])
          imf2 = sum(imf2_power)
          IMFr = imf2/imf1

          IMF1_rate = sum(EMD_signals["IMF1"][i])/len(EMD_signals["IMF1"][i])                         # IMF1 Rate
          IMF2_rate = sum(EMD_signals["IMF2"][i])/len(EMD_signals["IMF2"][i])                         # IMF2 Rate

          IMF1_stab = statistics.variance(EMD_signals["IMF1"][i])                      # IMF1 Stability
          IMF2_stab = statistics.variance(EMD_signals["IMF2"][i])                      # IMF2 Stability

          metric_results = metric_results._append({'Duration': phrases_new['Duration'][i], 'SBPr': SBR_3_0,'CNTR': CNTR,'IMFr': IMFr,'IMF1_rate': IMF1_rate,'IMF2_rate': IMF2_rate,'IMF1_stability': IMF1_stab,'IMF2_stability':IMF2_stab,'#':len(ESA_signals)}, ignore_index = True)

        result = metric_results.mean()
        result = pd.DataFrame(result).T
        results_all = pd.concat([results_all, result])

results_all

NF1032


FileNotFoundError: [Errno 2] No such file or directory: 'NF1032_frog.wav'